In [ ]:
import rasterio
import geopandas as gpd
import numpy as np
from skimage.graph.mcp import MCP_Geometric
from onstove.raster import *

In [ ]:
# This map represents the travel speed from this allocation process, expressed in units of minutes 
# required to travel one metre (Raster).
friction = r"C:\Proposals\CCA\Bimass_CODE_TEST_LAYERS\friction.tif"
# Startpoints (Raster)
forest_cover = r"C:\Proposals\CCA\Bimass_CODE_TEST_LAYERS\Forest.tif"
# Potential restriction areas (Polygon)
restrictions = r"C:\Proposals\CCA\Bimass_CODE_TEST_LAYERS\protected.shp"
restricted = r"C:\Proposals\CCA\Bimass_CODE_TEST_LAYERS\restricted_area.tif"
#new friction based on start points (Raster)
travel_time = r"C:\Proposals\CCA\Bimass_CODE_TEST_LAYERS\travel_time.tif"

In [ ]:
#Generated a restriction mask
mask_raster(forest_cover, restrictions, restricted, nodata=0, compression='NONE')
new_restrictions, new_restrictions_meta = align_raster(friction, restricted, method='nearest', compression='NONE')

In [ ]:
#Allign the forest raster with the friction raster
new_forest, new_forest_meta = align_raster(friction, forest_cover, method='nearest', compression='NONE')
new_forest = new_forest+new_restrictions
new_forest[new_forest == 1] = 0
new_forest[new_forest != 0] = 1
rows, cols = np.where(new_forest == 0)

In [ ]:
# Transform friction raster to km and hour instead of minutes and meter
with rasterio.open(friction) as fric:
    fric_meta = fric.meta
    fric_val = fric.read(1)
    fric_val = fric_val*1000/60

In [ ]:
# Create a new friction map by multiplying the firction map with the new forest cover map
new_fric = fric_val*new_forest

In [ ]:
# Minimum cost path betweenn all nodes
mcp = MCP_Geometric(new_fric, fully_connected=True)

In [ ]:
#Adding statvalues
pointlist = []
i = 0
for row in rows:
    positiions = [rows[i],cols[i]]
    pointlist.append(positiions)
    i = i+1

In [ ]:
#minimum cost
cumulative_costs, traceback = mcp.find_costs(starts=pointlist)
cumulative_costs[np.where(cumulative_costs==float('inf'))] = np.nan

In [ ]:
new_dataset = rasterio.open(travel_time, 'w', driver='GTiff',
                            height = src.shape[0], width = src.shape[1],
                            count=1, dtype="float64",
                            crs=src.crs, transform = src.transform)

new_dataset.write(cumulative_costs, 1)
new_dataset.close()